In [256]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
from matplotlib import pyplot
import matplotlib as mpl
import re
from collections import Counter
from nltk.tag import pos_tag
import nltk

%matplotlib inline

In [415]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_y_test = pd.read_csv('gender_submission.csv')
df_test['Survived'] = 3

In [258]:
df = pd.concat([df_train,df_test], axis=0)
df = df.reset_index()

In [259]:
print (len(df_train))
df_train.head()

891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [260]:
print (len(df_test))
df_test.head()

418


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,3
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,3
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,3
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,3


In [261]:
print (len(df))
df.head()

1309


,index,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0,A/5 21171
1,1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1,PC 17599
2,2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1,STON/O2. 3101282
3,3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1,113803
4,4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0,373450


In [262]:
print (len(df['SibSp'].unique()))
print (len(df['Parch'].unique()))
print (len(df['Ticket'].unique()))
print (len(df['Fare'].unique()))
print (len(df['Cabin'].unique()))
print (len(df['Embarked'].unique()))

7
8
929
282
187
4


In [263]:
print (df.groupby('Pclass')['Pclass'].count())
print (df.groupby('Sex')['Sex'].count())
print (df.groupby('SibSp')['SibSp'].count())
print (df.groupby('Parch')['Parch'].count())
print (df.groupby('Embarked')['Embarked'].count())

Pclass
1    323
2    277
3    709
Name: Pclass, dtype: int64
Sex
female    466
male      843
Name: Sex, dtype: int64
SibSp
0    891
1    319
2     42
3     20
4     22
5      6
8      9
Name: SibSp, dtype: int64
Parch
0    1002
1     170
2     113
3       8
4       6
5       6
6       2
9       2
Name: Parch, dtype: int64
Embarked
C    270
Q    123
S    914
Name: Embarked, dtype: int64


In [264]:
col_names = [ 'PassengerId', 'Pclass','Name','Sex','Age', 'SibSp', 'Parch', 'Ticket',  'Fare', 'Cabin', 'Embarked' ]

In [265]:
df.dtypes

index            int64
Age            float64
Cabin           object
Embarked        object
Fare           float64
Name            object
Parch            int64
PassengerId      int64
Pclass           int64
Sex             object
SibSp            int64
Survived         int64
Ticket          object
dtype: object

In [266]:
## Count NaN in each colums 
print ('[ Number of NaN in each Column ] ')
for i in range(len(col_names)):
    print (col_names[i], ":", len(df[col_names[i]]) - df[col_names[i]].count())
    
print ('[ Count 0 values in each Column ]')
print ("Ages :", len(df[(df['Age']==0)]))
print ("Fare :", len(df[(df['Fare']==0)]))

[ Number of NaN in each Column ] 
PassengerId : 0
Pclass : 0
Name : 0
Sex : 0
Age : 263
SibSp : 0
Parch : 0
Ticket : 0
Fare : 1
Cabin : 1014
Embarked : 2
[ Count 0 values in each Column ]
Ages : 0
Fare : 17


In [473]:
df.pivot_table(['Fare', 'Age'], index=['Pclass', 'Sex'] , aggfunc='mean')

Age        Fare
Pclass Sex                          
1      female  36.490920  109.412385
       male    39.285429   72.732680
2      female  27.566636   23.234827
       male    30.744355   20.628762
3      female  24.465569   15.324250
       male    27.106925   12.517228

In [474]:
df.pivot_table(['Fare', 'Age'], index=['Pclass', 'Sex'] , aggfunc='count')

Age  Fare
Pclass Sex              
1      female  144   144
       male    179   179
2      female  106   106
       male    171   171
3      female  216   216
       male    493   493

In [248]:
embarked_list = df.groupby('Embarked')['Embarked'].count().index.tolist()
for em in embarked_list:
    print ("-----------------------------------------")
    print ("Embarked "+str(em))
    print ("Mean fare : %.2f"%df[(df['Embarked']==em)]['Fare'].mean())
    print ("Class 1 : %.0f"%len(df[(df['Embarked']==em)&(df['Pclass']==1)]))
    print ("Class 2 : %.0f"%len(df[(df['Embarked']==em)&(df['Pclass']==2)]))
    print ("Class 3 : %.0f"%len(df[(df['Embarked']==em)&(df['Pclass']==3)]))

-----------------------------------------
Embarked C
Mean fare : 62.34
Class 1 : 141
Class 2 : 28
Class 3 : 101
-----------------------------------------
Embarked Q
Mean fare : 12.41
Class 1 : 3
Class 2 : 7
Class 3 : 113
-----------------------------------------
Embarked S
Mean fare : 27.42
Class 1 : 177
Class 2 : 242
Class 3 : 495


In [249]:
df.pivot_table('Fare', index=['Pclass', 'Embarked','Sex'] , aggfunc='mean')

Pclass  Embarked  Sex   
1       C         female    118.895949
                  male       94.622560
        Q         female     90.000000
                  male       90.000000
        S         female    101.069145
                  male       53.670756
2       C         female     27.003791
                  male       20.904406
        Q         female     12.350000
                  male       11.489160
        S         female     23.023118
                  male       20.073322
3       C         female     13.834545
                  male        9.775901
        Q         female      9.791968
                  male       10.979167
        S         female     18.083851
                  male       13.145977
Name: Fare, dtype: float64

In [250]:
df.pivot_table('Fare', index=['Pclass', 'Embarked','Sex'] , aggfunc='count')

Pclass  Embarked  Sex   
1       C         female     71
                  male       70
        Q         female      2
                  male        1
        S         female     69
                  male      108
2       C         female     11
                  male       17
        Q         female      2
                  male        5
        S         female     93
                  male      149
3       C         female     31
                  male       70
        Q         female     56
                  male       57
        S         female    129
                  male      365
Name: Fare, dtype: int64

In [251]:
df[(df['Sex']=='female')&(df['Pclass']==1)]['Embarked'].mode()[0]

'C'

In [335]:
def chg(dfx):
    
    dfp = df.pivot_table('Fare', index=['Pclass', 'Sex'] , aggfunc='mean')
    
    sub_class = [ x for x in dfp.index.get_level_values('Pclass')]
    sub_sex = [ x for x in dfp.index.get_level_values('Sex')]

    for i in range(len(dfx['Fare'])):
        
        if pd.isnull(dfx['Fare'].iloc[i]):
            dfx['Fare'].iloc[i] = float(dfp.loc[(dfp.index.get_level_values('Pclass')==dfx['Pclass'].iloc[i]) & (dfp.index.get_level_values('Sex')==dfx['Sex'].iloc[i])].values)
            

#    dfx['Fare'] = [float(dfp.loc[(dfp.index.get_level_values('Pclass')==dfx['Pclass']) & (dfp.index.get_level_values('Sex')==dfx['Sex'])].values) for x in dfx['Fare'] ]
    
    return dfx 


In [356]:
def clean_data(df):
    
    ## Clean Data 
    ## 'Age', 'Fare', 'Cabin', 'Embarked' 
    
    ## df['Age'] fill NaN to mean
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    
    ## df['Cabin'] fill NaN to 'Unknown'
    df['Cabin'] = df['Cabin'].fillna('Unknown')
    tmp_str = df['Cabin'].str.findall(r'[a-zA-Z]+').tolist()
    df['Cabin_deck']=[x[0] for x in tmp_str]
    tmp_str = df['Cabin'].str.findall(r'[0-9]+')
    df['Cabin_num'] = [x[0] if len(x) >0 else 0 for x in tmp_str]
    df['Cabin_count'] = [len(x) for x in tmp_str]
    
    ## df['Embarked'] fill NaN to 'C' <= Mode of class 1 & female 
    embarked_fill = df[(df['Sex']=='female')&(df['Pclass']==1)]['Embarked'].mode()[0]
    df['Embarked']=df['Embarked'].fillna(embarked_fill)
    
    ## df['Fare'] == 0 to NaN 
    df['Fare'] = df['Fare'].map(lambda x: np.nan if x==0 else x)
    
    df = chg(df)
#    df_pivot_fare_fill = df.pivot_table('Fare', index=['Pclass', 'Sex'] , aggfunc='mean')
#    df['Fare'] = df[['Fare', 'Pclass','Sex']].apply \
#    (lambda x: df_pivot_fare_fill[x['Pclass']] if pd.isnull(x['Fare']) else x['Fare'], axis=1 )
    
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']
    
    df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))
    
    def replace_titles(x):
        title=x['Title']
        if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
            return 'Mr'
        elif title in ['Countess', 'Mme']:
            return 'Mrs'
        elif title in ['Mlle', 'Ms']:
            return 'Miss'
        elif title =='Dr':
            if x['Sex']=='Male':
                return 'Mr'
            else:
                return 'Mrs'
        else:
            return title
    
    df['Title']=df.apply(replace_titles, axis=1)
    
    return df

df=clean_data(df)


In [357]:
def change_categ_to_numeric(df):
    
    df["Embarked"] = df["Embarked"].astype('category')
    df["Cabin_deck"] = df["Cabin_deck"].astype('category')
    df["Title"] = df["Title"].astype('category')
    df["Sex"] = df["Sex"].astype('category')

    
    df['nEmbark'] = df["Embarked"].cat.codes    
    df['nCabin_deck'] = df["Cabin_deck"].cat.codes
    df['nTitle'] = df["Title"].cat.codes
    df['nSex'] = df["Sex"].cat.codes
    
    return (df)
    
df =change_categ_to_numeric(df)

df.head()

,index,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,...,Survived,Ticket,Cabin_deck,Cabin_num,Cabin_count,Title,nEmbark,nCabin_deck,nTitle,nSex
0,0,22.0,Unknown,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,...,0,A/5 21171,Unknown,0,0,Mr,2,8,2,1
1,1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,...,1,PC 17599,C,85,1,Mrs,0,2,3,0
2,2,26.0,Unknown,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,...,1,STON/O2. 3101282,Unknown,0,0,Miss,2,8,1,0
3,3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,...,1,113803,C,123,1,Mrs,2,2,3,0
4,4,35.0,Unknown,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,...,0,373450,Unknown,0,0,Mr,2,8,2,1


In [460]:
## Get Train & Test Data 

def get_data(df):
    
    
    train_col_name = [ 'Age', 'Fare', 'Parch', 'Pclass', 'SibSp', 'nCabin_deck', 'nTitle', 'nSex']#, 'nEmbark', 'Cabin_count' ]
    truth_col_name = ['Survived']

    X = df[train_col_name][(df['Survived']<3)]
    y = (np.array(df[truth_col_name][(df['Survived']<3)]))
    y = y.reshape(len(y),)
    X_test = df[train_col_name][(df['Survived']==3)]
    
    print ("Train data len : %.0f"%len(X),"Test data len : %.0f"%len(X_test))
    y_test = list(df_y_test['Survived'])
    
    
    return X, y, X_test, y_test


In [461]:
X.head()

,Age,Fare,Parch,Pclass,SibSp,nEmbark,nCabin_deck,nTitle,nSex
0,22.0,7.2500,0,3,1,2,8,2,1
1,38.0,71.2833,0,1,1,0,2,3,0
2,26.0,7.9250,0,3,0,2,8,1,0
3,35.0,53.1000,0,1,1,2,2,3,0
4,35.0,8.0500,0,3,0,2,8,2,1


In [462]:
def decision_tree(df):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.tree import export_graphviz
    
    X_train, y_train, X_test, y_test = get_data(df)
    
    clf = DecisionTreeClassifier(max_depth=4, random_state=0)
    clf.fit(X_train, y_train)
    
    print ("#### Decision Tree ####")
    print ("Train Score : %.3f"%(100*clf.score(X_train, y_train))+"%")
    print ("Test Score : %.3f"%(100*clf.score(X_test, y_test))+"%")
    print ("Feature Importance", clf.feature_importances_)

decision_tree(df)

Train data len : 891 Test data len : 418
#### Decision Tree ####
Train Score : 84.400%
Test Score : 93.541%
Feature Importance [ 0.05103131  0.05918343  0.00749232  0.14227934  0.06663975  0.06582887
  0.03946037  0.56808461]


In [463]:
def logistic_classification(df):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing

    X_train, y_train, X_test, y_test = get_data(df)
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train) 
    X_test_scaled = scaler.transform(X_test)
    
    

    print ("#### Logistic Regression ####")
    C = [0.001, 0.01, 0.1, 1, 10, 100]
    
    for c in C :
        clf = LogisticRegression(C=c).fit(X_train_scaled, y_train)
        print ("--- C Value is : %.3f"%c)
        print ("Train Score : %.3f"%(100*clf.score(X_train_scaled, y_train))+"%")
        print ("Test Score : %.3f"%(100*clf.score(X_test_scaled, y_test))+"%")

logistic_classification(df)

Train data len : 891 Test data len : 418
#### Logistic Regression ####
--- C Value is : 0.001
Train Score : 76.880%
Test Score : 87.799%
--- C Value is : 0.010
Train Score : 78.900%
Test Score : 93.301%
--- C Value is : 0.100
Train Score : 79.910%
Test Score : 93.062%
--- C Value is : 1.000
Train Score : 80.022%
Test Score : 92.584%
--- C Value is : 10.000
Train Score : 80.022%
Test Score : 92.584%
--- C Value is : 100.000
Train Score : 80.022%
Test Score : 92.584%


In [444]:
def SVM_classification(df):
    from sklearn.svm import LinearSVC
    from sklearn import preprocessing

    X_train, y_train, X_test, y_test = get_data(df)
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train) 
    X_test_scaled = scaler.transform(X_test)
    
    C = [0.001, 0.01, 0.1, 1, 10, 100]
    
    print ("#### SVM Classifier ####")
    
    for c in C :
        clf = LinearSVC(C=c).fit(X_train_scaled, y_train)

        print ("--- C Value is : %.3f"%c)
        print ("Train Score : %.3f"%(100*clf.score(X_train_scaled, y_train))+"%")
        print ("Test Score : %.3f"%(100*clf.score(X_test_scaled, y_test))+"%")

SVM_classification(df)

Train data len : 891 Test data len : 418
#### SVM Classifier ####
--- C Value is : 0.001
Train Score : 78.676%
Test Score : 92.584%
--- C Value is : 0.010
Train Score : 79.574%
Test Score : 94.737%
--- C Value is : 0.100
Train Score : 80.022%
Test Score : 94.498%
--- C Value is : 1.000
Train Score : 80.135%
Test Score : 94.498%
--- C Value is : 10.000
Train Score : 79.574%
Test Score : 96.411%
--- C Value is : 100.000
Train Score : 79.237%
Test Score : 90.431%


In [464]:
def RadndomForest_classification(df):
    from sklearn.ensemble import RandomForestClassifier

    X_train, y_train, X_test, y_test = get_data(df)
    
    est = [2, 5, 10,12,14,15,16, 20]
    
    print ("#### Random Forest Classifier ####")
    
    for c in est :
        clf = RandomForestClassifier(n_estimators=c).fit(X_train, y_train)

        print ("--- Est Value is : %.3f"%c)
        print ("Train Score : %.3f"%(100*clf.score(X_train, y_train))+"%")
        print ("Test Score : %.3f"%(100*clf.score(X_test, y_test))+"%")
        print ("Feature Importance : ", clf.feature_importances_)

RadndomForest_classification(df)

Train data len : 891 Test data len : 418
#### Random Forest Classifier ####
--- Est Value is : 2.000
Train Score : 90.685%
Test Score : 83.014%
Feature Importance :  [ 0.26096475  0.22645932  0.02124058  0.04598355  0.0437665   0.07850384
  0.08034033  0.24274112]
--- Est Value is : 5.000
Train Score : 95.286%
Test Score : 81.100%
Feature Importance :  [ 0.23014027  0.22601476  0.03802421  0.08108923  0.06171201  0.05936515
  0.11452267  0.1891317 ]
--- Est Value is : 10.000
Train Score : 97.306%
Test Score : 82.536%
Feature Importance :  [ 0.22201307  0.24308647  0.03331997  0.08468366  0.04920716  0.07416312
  0.14953712  0.14398944]
--- Est Value is : 12.000
Train Score : 97.643%
Test Score : 82.536%
Feature Importance :  [ 0.23931617  0.24504812  0.03165385  0.06474076  0.05061211  0.05963982
  0.08336147  0.22562769]
--- Est Value is : 14.000
Train Score : 97.419%
Test Score : 83.254%
Feature Importance :  [ 0.20814673  0.26388137  0.03514921  0.0643159   0.04854987  0.06447592
  

In [468]:
def SVM_Kernel_classification(df):
    from sklearn.svm import SVC
    from sklearn import preprocessing

    X_train, y_train, X_test, y_test = get_data(df)
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train) 
    X_test_scaled = scaler.transform(X_test)
    
    C = [0.1, 1, 10, 100]
    G = [0.1, 1, 10]
    
    print ("#### SVM kernel Classifier ####")
    
    for c in C :
        g = 0.1
        clf = SVC(kernel='rbf', C=c, gamma=g).fit(X_train_scaled, y_train)

        print ("--- C Value is : %.3f"%c, 'Gamma value is : %.3f'%g)
        print ("Train Score : %.3f"%(100*clf.score(X_train_scaled, y_train))+"%")
        print ("Test Score : %.3f"%(100*clf.score(X_test_scaled, y_test))+"%")

SVM_Kernel_classification(df)

Train data len : 891 Test data len : 418
#### SVM kernel Classifier ####
--- C Value is : 0.100 Gamma value is : 0.100
Train Score : 83.165%
Test Score : 94.976%
--- C Value is : 1.000 Gamma value is : 0.100
Train Score : 83.838%
Test Score : 94.737%
--- C Value is : 10.000 Gamma value is : 0.100
Train Score : 85.634%
Test Score : 90.431%
--- C Value is : 100.000 Gamma value is : 0.100
Train Score : 87.542%
Test Score : 87.321%


In [472]:
def MLP_classification(df):
    from sklearn.neural_network import MLPClassifier
    from sklearn import preprocessing

    X_train, y_train, X_test, y_test = get_data(df)
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train) 
    X_test_scaled = scaler.transform(X_test)
    
    C = [0.1, 1, 10, 100]
    G = [0.1, 1, 10]
    layer = [[5], [10], [20,10], [ 30, 10],[100, 20], [100,50], [50,30,20],  [30,20,10], [ 30, 20, 10, 5]]
    
    print ("#### MLP Classifier ####")
    
    for l in layer : 
        clf = MLPClassifier(solver='lbfgs', activation = 'relu', random_state=0, hidden_layer_sizes=l).fit(X_train_scaled, y_train)
        print ("----- Layer : ", l)
        print ("Train Score : %.3f"%(100*clf.score(X_train_scaled, y_train))+"%")
        print ("Test Score : %.3f"%(100*clf.score(X_test_scaled, y_test))+"%")

MLP_classification(df)

Train data len : 891 Test data len : 418
#### MLP Classifier ####
----- Layer :  [5]
Train Score : 83.951%
Test Score : 87.799%
----- Layer :  [10]
Train Score : 86.420%
Test Score : 87.560%
----- Layer :  [20, 10]
Train Score : 90.685%
Test Score : 84.928%
----- Layer :  [30, 10]
Train Score : 90.797%
Test Score : 85.167%
----- Layer :  [100, 20]
Train Score : 92.705%
Test Score : 82.297%
----- Layer :  [100, 50]
Train Score : 92.817%
Test Score : 82.297%
----- Layer :  [50, 30, 20]
Train Score : 92.368%
Test Score : 80.622%
----- Layer :  [30, 20, 10]
Train Score : 90.460%
Test Score : 84.689%
----- Layer :  [30, 20, 10, 5]
Train Score : 90.572%
Test Score : 84.211%


In [13]:
def show_histogram(x, bins):
    plt.hist(x, bins = bins, color='blue')
    plt.show()


In [345]:
def substrings_in_string(big_string, substrings):
    import string
    for substring in substrings:
#        if string.find(big_string, substring) != -1:
        if big_string.find(substring) != -1:
            return substring
    print (big_string)
    return np.nan